# Classification With Decision Trees, From Start to Finish

Author: ***Soroush Ghaderi***

In this lesson we will use **scikit-learn** and **Cost Complexity Pruning** to build **Classification Tree** which uses continues and categorical data from [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php) to predict whether or not a paitent has [heart disease](https://archive.ics.uci.edu/ml/datasets/heart+disease).